In [1]:
import os, sys
import json
import socket
import requests

from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QTime

In [2]:
class ClientAgent:
    def __init__(self, path):
        self.path = path
        
    def web_request(self, method_name, url, dict_data, is_urlencoded=True):
        # Web GET or POST request를 호출 후 그 결과를 dict형으로 반환
        method_name = method_name.upper()  # 메소드이름을 대문자로 바꾼다
        if method_name not in ('GET', 'POST'):
            raise Exception('method_name is GET or POST plz...')
            
        if method_name == 'GET':  # GET방식인 경우
            response = requests.get(url=url, params=dict_data)
        
        elif method_name == 'POST':  # POST방식인 경우
            if is_urlencoded is True:
                response = requests.post(url=url, data=dict_data, headers={'Content-Type': 'application/x-www-form-urlencoded'})
            else:
                response = requests.post(url=url, data=json.dumps(dict_data), headers={'Content-Type': 'application/json'})
        dict_meta = {'status_code': response.status_code, 'ok': response.ok, 'encoding': response.encoding,
                     'Content-Type': response.headers['Content-Type']}
        if 'json' in str(response.headers['Content-Type']):  # JSON 형태인 경우
            return {**dict_meta, **response.json()}
        else:  # 문자열 형태인 경우
            return {**dict_meta, **{'text': response.text}}

    def callFirst(self):
        url = self.path+"first"
        response = self.web_request(method_name='GET', url=url, dict_data=None)
        return response["text"]
    
    def callNext(self, userActions):
        data = {'data': userActions}
        url = self.path + "next"
        response = self.web_request(method_name='POST', url=url, dict_data=data)
        return response["text"]
    
    def callPressed(self, content):
        query = {}
        query["id"] = content.getId()
        query["flag"] = content.getLikeFlag()
        url = self.path + "press"
        response = self.web_request(method_name='POST', url=url, dict_data=query)
        
    def callConsumed(self, content, time):
        query = {}
        query["id"] = content.getId()
        query["time"] = time
        url = self.path + "consum"
        response = self.web_request(method_name='POST', url=url, dict_data=query)
    
    @staticmethod
    def Test():
        ca = ClientAgent(path = 'http://127.0.0.1:5000/')
        response = ca.callFirst()
        print(response)
        response = ca.callNext("hello")
        print(response)
        response = ca.callNext("hello")
        print(response)
        response = ca.callNext("hello")
        print(response)
        response = ca.callNext("hello")
        print(response)
        response = ca.callNext("hello")
        print(response)

In [3]:
class Content:
    
    MIN_CONSUMEPTION_TIME = 5.0
    MAX_CONSUMEPTION_TIME = 30.0
    
    def __init__(self, id, consumedTime, likeFlag, tags):
        self.id = id
        self.consumedTime = consumedTime
        self.likeFlag = likeFlag
        self.tags = tags
    
    def getId(self):
        return self.id
    
    def getLikeFlag(self):
        return self.likeFlag
        
    def getConsumedTime(self):
        return self.consumedTime
    
    def isConsumed(self):
        return (self.MIN_CONSUMEPTION_TIME<=self.consumedTime) & (self.consumedTime<=self.MAX_CONSUMEPTION_TIME)
    
    def getTags(self):
        return self.tags
    
    def addConsumedTime(self, time):
        if time < self.MIN_CONSUMEPTION_TIME:
            return
        else:
            self.stayTime += time
            
        if self.consumedTime > self.MAX_CONSUMEPTION_TIME:
            self.consumedTime = self.MAX_CONSUMEPTION_TIME
    
    def initConsumedTime(self):
        self.sonsumedTime = 0
    
    def like(self):
        self.likeFlag = True

    def cancel(self):
            self.likeFlag = False

            
    def print(self):
        print("__________________________________________")
        print("# create new content and show")
        print("id:       %s"%(self.id))
        print("consumeTime: %f"%(self.consumedTime))
        print("likeFalg: %s"%("True" if self.likeFlag else "False"))
        print("tags:     %s"%(str(self.tags)))
        

    @staticmethod
    def Test(self):
        content = Content("animal_0.jpg", 10.0, False, ["animal"])
        content.print()

In [4]:
class ContentManager:
    def __init__(self):
        self.idx = -1
        self.ca = ClientAgent(path = 'http://127.0.0.1:5000/')
        self.contents = []
        
    def bring(self):
        if len(self.contents) == 0:
            newContents = self.ca.callFirst()
            self.idx = 0
        else:
            newContents = self.ca.callNext("")
        
        newContents = json.loads(newContents)

        for contentId in newContents.keys():
            c = newContents[contentId]
            c = Content(c["id"], c["consumed_time"], c["like_flag"], c["tags"])
            self.contents.append(c)

    def getCurContent(self):
        return self.contents[self.idx]
    
    def next(self):
        self.idx += 1
        if len(self.contents) <= self.idx:
            self.idx = len(self.contents)-1
            self.bring()
            
    def before(self):
        self.idx -= 1;
        if self.idx < 0:
            self.idx = 0
            
    def print(self):
        for content in self.contents: 
            content.print()
        
    @staticmethod
    def Test():
        cm = ContentManager()
        cm.bring()
        cm.print()
        cm.bring()
        cm.print()

In [5]:
from datetime import datetime

class StopWatch:
    def __init__(self):
        self.reset()

    def reset(self):
        self.start_datetime = None
        self.end_datetime = None
        
    def start(self):
        self.reset()
        self.start_datetime = datetime.now()
        
    def stop(self):
        self.end_datetime = datetime.now()
        
    def getTime(self):
        return (self.end_datetime - self.start_datetime).total_seconds() 

In [6]:
class MyApp(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        self.stopWatch = StopWatch()
        self.ca = ClientAgent(path = 'http://127.0.0.1:5000/')
        self.cm = ContentManager()
        self.cm.bring()
        self.initUI()
        

    def initUI(self):
        self.setObjectName("Iphone_Instagram")
        self.resize(448, 875)
        self.setFixedSize(448, 875)
        self.setStyleSheet("background-color: rgb(225, 225, 225);")

        # ********** ********** label_2 ********** **********
        self.label_2 = QtWidgets.QLabel(self)
        self.label_2.setGeometry(QtCore.QRect(-10, 0, 461, 881))
        self.label_2.setStyleSheet("image: url(ui_image/iphone_instagram2.png);")
        self.label_2.setText("")
        self.label_2.setObjectName("label_2")

        # ********** ********** label ********** **********
        self.mainLabel = QtWidgets.QLabel(self)
        self.mainLabel.setGeometry(QtCore.QRect(40, 230, 361, 361))
        self.mainLabel.setStyleSheet("background-color: rgb(255, 255, 255);")
        self.mainLabel.setText("")
        self.mainLabel.setObjectName("mainLabel")

        # ********** ********** NextButton ********** **********
        self.pushButton = QtWidgets.QPushButton(self)
        self.pushButton.setGeometry(QtCore.QRect(420, 860, 21, 20))
        self.pushButton.setStyleSheet("background-color: rgb(255, 255, 255);\n"
                                      "image: url(ui_image/Button.JPG);")
        self.pushButton.setText("")
        self.pushButton.setObjectName("pushButton")
        self.pushButton.clicked.connect(self.showNextImage)
        
        # ********** ********** BeforeButton ********** **********
        self.pushButton = QtWidgets.QPushButton(self)
        self.pushButton.setGeometry(QtCore.QRect(20, 860, 21, 20))
        self.pushButton.setStyleSheet("background-color: rgb(255, 255, 255);\n"
                                      "image: url(ui_image/Button.JPG);")
        self.pushButton.setText("")
        self.pushButton.setObjectName("pushButton")
        self.pushButton.clicked.connect(self.showBeforeImage)
        

        # ********** ********** pushButton2 ********** **********
        self.pushButton_2 = QtWidgets.QPushButton(self)
        self.pushButton_2.setGeometry(QtCore.QRect(42, 606, 41, 31))
        self.pushButton_2.setStyleSheet("background-color: rgb(255, 255, 255);\n"
                                        "image: url(ui_image/heart.png);")
        self.pushButton_2.setText("")
        self.pushButton_2.setObjectName("ui_image/pushButton_2")
        self.pushButton_2.clicked.connect(self.pushHeart)

        # ********** ********** Execute ********** **********
        self.center()  # ** 창을 화면의 정 가운데에 위치 **
        self.show()
        self.stopWatch.start()
        self.showContent()
        
    def showContent(self):
        content = self.cm.getCurContent()
        filename = content.getId()
        filepath = "image/%s"%(filename)
        image = QtGui.QImage(filepath)
        image = self.setSize(image, 361, 361)
        self.eraseMainLabel()
        self.mainLabel.setPixmap(QtGui.QPixmap.fromImage(image))
        
        if content.getLikeFlag():
            self.pushButton_2.setStyleSheet("background-color: rgb(255, 255, 255);\n"
                                            "image: url(ui_image/heart2.png);")
        else:
            self.pushButton_2.setStyleSheet("background-color: rgb(255, 255, 255);\n"
                                            "image: url(ui_image/heart.png);")

    def showNextImage(self):
        self.stopWatch.stop()
        time = self.stopWatch.getTime()
        self.stopWatch.start()
        self.ca.callConsumed(self.cm.getCurContent(), time)
        self.cm.next()
        self.showContent()

        
    def showBeforeImage(self):
        self.stopWatch.stop()
        time = self.stopWatch.getTime()
        self.stopWatch.start()
        self.ca.callConsumed(self.cm.getCurContent(), time)
        self.cm.before()
        self.showContent()


    def eraseMainLabel(self):
        image = QtGui.QImage("ui_image/Button.JPG")
        image = self.setSize(image, 361, 361)
        self.mainLabel.setPixmap(QtGui.QPixmap.fromImage(image))

    def setSize(self, image, width, height):
        image = image.scaledToWidth(width)
        image = image.scaledToHeight(height)
        return image

    def pushHeart(self):
        content = self.cm.getCurContent()
        if content.getLikeFlag():
            content.cancel()
        else:
            content.like()
            
        self.showContent()
        self.ca.callPressed(content)
        
    def center(self):
        ct = self.frameGeometry()
        ct2 = QtWidgets.QDesktopWidget().availableGeometry().center()
        ct.moveCenter(ct2)
        self.move(ct.topLeft())

In [7]:
if __name__ == "__main__":
    app = QtWidgets.QApplication(sys.argv)
    ex = MyApp()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\hooni\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
